In [9]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded

if not is_dbenv_loaded():
    load_dbenv()

import pymatgen
from aiida.work.run import run
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.base import  Str, Bool, Float, Int
from aiida_defects.formation_energy.makovpayne import MakovPayneCorrection
from aiida.orm import load_node


inputfile='batio3_222.cif'

structure_mg = pymatgen.Structure.from_file(str(inputfile))
structure_sd = StructureData(pymatgen=structure_mg)
codename="pw_6.0_eb@ubelix2"
pseudo_family='SSSP'
#defect_position = ArrayData()
#defect_position.set_array("position", np.array([0., 0., 0.]))
defect_charge= 2.
epsilon_r =  2.

options={
        'resources': {
            'num_machines': 1,
            #'num_mpiprocs_per_machine': 1,
        },
         'max_wallclock_seconds' : 600,
         #'custom_scheduler_commands' : u"#SBATCH --partition=all",
         #'custom_scheduler_commands' : u"#SBATCH --account=dcb",
         'custom_scheduler_commands' : u"#SBATCH --partition=empi",

        }

settings={}

kpoints = KpointsData()
kpoints.set_kpoints_mesh([8, 8, 8])


parameters = {
        'CONTROL': {
            'restart_mode': 'from_scratch',
            #'tstress': True,
            'tprnfor' : True,
            'etot_conv_thr' : 1.0e-6,
             'forc_conv_thr' : 1.0e-3,
        },
        'SYSTEM': {
            'ecutwfc': 60.,
            'ecutrho': 480.,
            'occupations' : 'smearing',
            'degauss' : 0.01,
            #'starting_magnetization' : starting_magnetization,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-8,
            'mixing_beta' : 0.6,
            'startingwfc' : 'atomic',
        },

    }


outputs=run(MakovPayneCorrection,
            bulk_structure=structure_sd,
            code_pw=Str(codename),
            pseudo_family=Str(pseudo_family),
            kpoints=kpoints,
            parameters=ParameterData(dict=parameters),
            settings=ParameterData(dict=settings),
            options=ParameterData(dict=options),
            epsilon_r=Float(epsilon_r),
            defect_charge=Float(defect_charge)
           )


12/04/2018 03:06:25 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [154546|MakovPayneCorrection|run_pw_H_structure]: Launching PwBaseWorkChain for the calculation of the first order Makov Payne Correction. pk value 154550
12/04/2018 03:08:11 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [154546|MakovPayneCorrection|compute_correction]: Makov Payne Correction -4.16476609747
12/04/2018 03:08:11 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [154546|MakovPayneCorrection|compute_correction]: MakovPayneCorrection workchain completed succesfully


In [10]:
print outputs

{'Makov_Payne_Correction': -4.1647660974693}
